In [1]:
import heapq
import itertools
import time

import gurobipy as grb
import math

import networkx as nx
from gurobipy import GRB
from tspData.tsp20_0 import points
from itertools import combinations


from const import *
from geometry import *
from graph import *

def distance(city1, city2):
    Differ = (points[city1][0] - points[city2][0], points[city1][1] - points[city2][1])
    return math.sqrt(Differ[0] * Differ[0] + Differ[1] * Differ[1])

# ipTSP only requires nodes;edges;fml;outputFlag

def ipTSP(
    nodes:      "parameter with the following form(a dictionary) \
                    {\
                        nodeID1: {'loc1': (x1, y1)}, \
                        nodeID2: {'loc2': (x2, y2)}, \
                        ... \
                    }" = None, 
    edges:      "Here we require a matrix restoring distances between nodes \
                 1) String (default) 'Euclidean' or \
                 2) String 'LatLon' or \
                 3) String 'Grid' or \
                 4) Dictionary {(nodeID1, nodeID2): dist, ...}" = "Euclidean",
    edgeArgs:   "If choose 'Grid' as tau option, we need the following dictionary \
                {\
                    'colRow': (numCol, numRow),\
                    'barriers': [(coordX, coordY), ...], \
                }" = None,
    depotID:    "Depot ID, truck visiting sequence will start from and end with this nodeID" = 0, #0 is default
    nodeidx:    "Here nodeidx stands for loc of {loc:(x,y)} \
                 1) String (default) 'All', or \
                 2) A list of node IDs" = 'All',
    serviceTime: "Service time spent on each customer (will be added into travel matrix)" = 0, #0 is default
    fml:        "1) String (default) 'DFJ_Lazy' or \
                 2) String 'DFJ_PlainLoop' or \
                 3) String 'MTZ' or \
                 4) String 'MultiCommodityFlow' or \
                 5) String 'ShortestPath' or \
                 6) String 'QAP'" = 'DFJ_Lazy',
    timeLimit:  "1) Double, in seconds or \
                 2) (default) None, no time limit" = None,
    gapTolerance: "1) Double, Stopping gap, or \
                 2) (default) None, no gap limit" = None,
    outputFlag: "Boolean, True if export the gurobi logs" = False
    ) -> "Exact solution for TSP":

    #定义 nodeidx==========================================================
    if (type(nodeidx) is not list): 
        if (nodeidx == 'All'):
            nodeidx = []
            for i in nodes:
                nodeidx.append(i)
        else:
            print(ERROR_INCOR_NODEIDS)
            return

    # 定义 tau ==============================================================
    tau = {}
    tau = edges 
    serviceTime = 0

    # Gurobi initialize =======================================================
    n = len(nodeidx)
    TSP = grb.Model('TSP')
    # Subroutines for different formulations ==================================
    def _ipTSPLazyCuts():
        # 建立决策变量 --------------------------------------------------
        # {(0, 1): <gurobi.Var x_0_1>, (0, 2): <gurobi.Var x_0_2>, ... }
        x = {}
        for i in range(n):
            for j in range(n):
                if (i != j):
                    x[i, j] = TSP.addVar(
                        vtype=grb.GRB.BINARY,   # Bibary 表明是0-1变量
                        obj=tau[nodeidx[i], nodeidx[j]],    # c_{ij} 代表距离
                        name='x_%s_%s' % (i, j)
                    )

        # TSP 目标函数 ----------------------------------------------
        TSP.modelSense = grb.GRB.MINIMIZE
        TSP.Params.lazyConstraints = 1
        TSP.update()

        # 两个约束 --------------------------------------------------
        for i in range(n):
            TSP.addConstr(grb.quicksum(x[i, j] for j in range(n) if i != j) == 1, name='leave_%s' % i)
            TSP.addConstr(grb.quicksum(x[j, i] for j in range(n) if i != j) == 1, name='enter_%s' % i)

        
        # 子环消除 ------------------------------------------------
        TSP._x = x
        #Lazy cut
        def subtourelim(model, where):
            if where == grb.GRB.Callback.MIPSOL:
                vals = model.cbGetSolution(model._x)
                selected = grb.tuplelist((i, j) for i, j in model._x.keys() if x_sol[i, j] > 0.5)
                components = getGraphComponents(selected)
                for component in components:
                    if len(component) < n:
                        model.cbLazy(grb.quicksum(
                            x[i,j] for i in component for j in component if i != j) <= len(component) - 1)

        # TSP with callback ---------------------------------------------------
        TSP.optimize(subtourelim)

        # Reconstruct solution ------------------------------------------------
        ofv = None
        node = []
        edge = []
        if (TSP.status == grb.GRB.status.OPTIMAL):
            ofv = TSP.getObjective().getValue()
            for i,j in x:
                if (x[i,j].x > 0.5):
                    edge.append([i,j])
            currentTime = 0
            currentNode = 0
            node.append(nodeidx[currentNode])
            while (len(edge) > 0):
                for i in range(len(edge)):
                    if (edge[i][0] == currentNode):
                        currentNode = edge[i][1]
                        node.append(nodeidx[currentNode])
                        edge.pop(i)    
                        break
            gap = 0
            lb = ofv
            ub = ofv
            runtime = TSP.Runtime
        elif (TSP.status == grb.GRB.status.TIME_LIMIT):
            ofv = None
            node = []
            gap = TSP.MIPGap
            lb = TSP.ObjBoundC
            ub = TSP.ObjVal
            runtime = TSP.Runtime

        vals = TSP.getAttr('x', x)
        selected = grb.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
        tour = getGraphComponents(selected)
        print(*tour,len(*tour))
        return {
            'ofv': ofv,
            'seq': node,
            'gap': gap,
            'lowerBound': lb,
            'upperBound': ub,
            'runtime': runtime
        }
    
    #User cut======================================================================================================
    
    def _ipTSPUserCuts():
        # 决策变量 --------------------------------------------------
        x = {}
        for i in range(n):
            for j in range(n):
                if (i !=j):
                    x[i, j] = TSP.addVar(
                        vtype=grb.GRB.BINARY,  
                        obj=tau[nodeidx[i], nodeidx[j]],  
                        name='x_%s_%s' % (i, j)
                    )

        # TSP 目标函数 ----------------------------------------------
        TSP.modelSense = grb.GRB.MINIMIZE
        TSP.Params.lazyConstraints = 1
        # TSP.Params.LogToConsole = 1
        TSP.Params.PreCrush = 1
        TSP.update()

        # 约束条件 --------------------------------------------------
        for i in range(n):
            TSP.addConstr(grb.quicksum(x[i, j] for j in range(n) if i != j) == 1.0, name = 'leave_%s' % i)
            TSP.addConstr(grb.quicksum(x[j, i] for j in range(n) if i != j) == 1.0, name = 'enter_%s' % i)

        # 子环消除: min-cut ---------------------------------------
        TSP._x = x

        def mincut(sol):
            def combine(G, s, t):  # 将点s并入点t where s>t
                # 包含s的边全部并入t
                edges = G.edges
                s_neibors = G.adj[s].copy()
                for neibor in s_neibors:
                    if neibor != t:
                        capacity_n_t = G.get_edge_data(neibor, t)['capacity'] if (neibor, t) in edges else 0
                        G.add_edge(neibor, t, capacity=capacity_n_t + G.get_edge_data(neibor, s)['capacity'])
                    G.remove_edge(neibor, s)
                G.remove_node(s)

            G = nx.Graph()
            for (start, end) in sol:
                G.add_edge(start, end, capacity=sol[(start, end)])
            V = set(G.nodes)
            value_partition = {}
            while len(G.nodes) >= 2:
                s = list(G.nodes)[-2]
                t = list(G.nodes)[-1]
                cut_value, partition = nx.minimum_cut(G, s, t)
                value_partition[cut_value] = partition
                combine(G, t, s)
            min_cut_value = min(value_partition.keys())
            best_partition = value_partition[min_cut_value]
            best_partition = (best_partition[0], V - best_partition[0])

            return min_cut_value, best_partition

        def sol_convert(rel):
            sol = {}
            for key in rel:
                if key[0] < key[1]:
                    rel[key] = rel[key] + rel[(key[1], key[0])]
                    if rel[key] > 0:
                        sol[key] = rel[key]
            return sol

        def subtourelim(model, where):
            if where == grb.GRB.Callback.MIPSOL:
                x_sol = model.cbGetSolution(model._x)
                selected = grb.tuplelist((i, j) for i, j in model._x.keys() if x_sol[i, j] > 0.9)  
                components = getGraphComponents(selected)
                for component in components:
                    if len(component) < n:
                        model.cbLazy(grb.quicksum(
                            x[i, j] for i in component for j in component if i != j) <= len(component) - 1)

            if where == GRB.Callback.MIPNODE:  
                status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
                if status == GRB.OPTIMAL:   # 线性松弛问题最优解
                    rel = model.cbGetNodeRel(model._x)  
                    # 通过最小割分成不连通子图
                    sol = sol_convert(rel)
                    cut_value, partition = mincut(sol)
                    # 子环存在如果权值之和小于2
                    if cut_value < 1.9:
                        model.cbCut(grb.quicksum(
                            x[i, j] for i in partition[0] for j in partition[0] if i != j) <= len(partition[0]) - 1)

        # TSP with callback ---------------------------------------------------
        TSP.optimize(subtourelim)

        # Reconstruct solution ------------------------------------------------
        ofv = None
        node = []
        edge = []
        if TSP.status == grb.GRB.status.OPTIMAL:
            ofv = TSP.getObjective().getValue()
            for i, j in x:
                if x[i, j].x > 0.5:
                    edge.append([i, j])
            currentNode = 0
            currentTime = 0
            node.append(nodeidx[currentNode])
            while len(edge) > 0:
                for i in range(len(edge)):
                    if edge[i][0] == currentNode:
                        currentNode = edge[i][1]
                        node.append(nodeidx[currentNode])
                        edge.pop(i)
                        break
            gap = 0
            lb = ofv
            ub = ofv
            runtime = TSP.Runtime
        elif (TSP.status == grb.GRB.status.TIME_LIMIT):
            ofv = None
            node = []
            gap = TSP.MIPGap
            lb = TSP.ObjBoundC
            ub = TSP.ObjVal
            runtime = TSP.Runtime

        vals = TSP.getAttr('x', x)
        selected = grb.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
        tour = getGraphComponents(selected)
        print(*tour,len(*tour))
        
        return {
            'ofv': ofv,
            'seq': node,
            'gap': gap,
            'lowerBound': lb,
            'upperBound': ub,
            'runtime': runtime
        }

    # Solve by different formulations =========================================
    tsp=None
    tsp=_ipTSPUserCuts()
    if (tsp != None):
        tsp['fml'] = fml

    if len(tsp['seq']) == 0:
        return tsp
    # Fix the sequence to make it start from the depot ========================
    startIndex = None
    seq = [i for i in tsp['seq']]
    truckSeq = []
    for k in range(len(seq)):
        if (seq[k] == depotID):
            startIndex = k
    if (startIndex <= len(seq) - 1):
        for k in range(startIndex, len(seq) - 1):
            truckSeq.append(seq[k])
    if (startIndex >= 0):
        for k in range(0, startIndex):
            truckSeq.append(seq[k])
    truckSeq.append(depotID)
    tsp['seq'] = truckSeq

    # Add service time info ===================================================
    tsp['serviceTime'] = serviceTime

    return tsp

a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True) 



Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-02
Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0x61b68f0c
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 3.238422e+00, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.23842    0   22   

{'ofv': 3.748306438525786,
 'seq': [0,
  2,
  15,
  13,
  5,
  3,
  7,
  11,
  19,
  6,
  1,
  17,
  9,
  4,
  14,
  18,
  8,
  16,
  12,
  10,
  0],
 'gap': 0,
 'lowerBound': 3.748306438525786,
 'upperBound': 3.748306438525786,
 'runtime': 0.09973335266113281,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [2]:
from tspData.tsp20_1 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0xa07f7771
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 2.778085e+00, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.46528    0   26          -    3.46528      -     -    0s
*    0     0               0       3.5137257    3.51

{'ofv': 3.513725681938508,
 'seq': [0,
  7,
  4,
  12,
  5,
  17,
  14,
  1,
  3,
  13,
  15,
  9,
  16,
  2,
  6,
  18,
  11,
  8,
  19,
  10,
  0],
 'gap': 0,
 'lowerBound': 3.513725681938508,
 'upperBound': 3.513725681938508,
 'runtime': 0.05485343933105469,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [3]:
from tspData.tsp20_2 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0xb386369d
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 3.565270e+00, 59 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.6053089    3.60531  0.00%     -    0s

Cutting planes:
  Lazy constraints: 11

Explored 1 

{'ofv': 3.6053089491031547,
 'seq': [0,
  1,
  18,
  12,
  17,
  8,
  11,
  15,
  14,
  4,
  5,
  2,
  16,
  7,
  3,
  6,
  9,
  13,
  10,
  19,
  0],
 'gap': 0,
 'lowerBound': 3.6053089491031547,
 'upperBound': 3.6053089491031547,
 'runtime': 0.04487800598144531,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [4]:
from tspData.tsp40_0 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 1560 columns and 3120 nonzeros
Model fingerprint: 0xa2942fff
Variable types: 0 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 80 rows, 1560 columns, 3120 nonzeros
Variable types: 0 continuous, 1560 integer (1560 binary)

Root relaxation: objective 4.458154e+00, 98 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.45815    0   40          -    4.45815      -     -    0s
*    0     0               0       5.0283958

{'ofv': 5.028395819053002,
 'seq': [0,
  20,
  37,
  34,
  3,
  31,
  32,
  38,
  19,
  14,
  36,
  12,
  9,
  16,
  33,
  21,
  18,
  8,
  2,
  1,
  26,
  29,
  10,
  17,
  24,
  15,
  28,
  35,
  4,
  13,
  30,
  22,
  27,
  5,
  6,
  11,
  23,
  25,
  39,
  7,
  0],
 'gap': 0,
 'lowerBound': 5.028395819053002,
 'upperBound': 5.028395819053002,
 'runtime': 0.18921470642089844,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [5]:
from tspData.tsp40_1 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 1560 columns and 3120 nonzeros
Model fingerprint: 0xcfcfd600
Variable types: 0 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 80 rows, 1560 columns, 3120 nonzeros
Variable types: 0 continuous, 1560 integer (1560 binary)

Root relaxation: objective 5.272945e+00, 128 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.27295    0   38          -    5.27295      -     -    0s
     0     0    5.55275    0   26          

{'ofv': 5.574060657101232,
 'seq': [0,
  4,
  11,
  26,
  2,
  3,
  35,
  10,
  24,
  20,
  23,
  6,
  38,
  31,
  36,
  1,
  19,
  14,
  27,
  15,
  22,
  34,
  5,
  37,
  39,
  32,
  12,
  18,
  29,
  25,
  33,
  13,
  16,
  17,
  8,
  30,
  7,
  28,
  21,
  9,
  0],
 'gap': 0,
 'lowerBound': 5.574060657101232,
 'upperBound': 5.574060657101232,
 'runtime': 5.661035537719727,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [6]:
from tspData.tsp40_2 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 1560 columns and 3120 nonzeros
Model fingerprint: 0x935a6777
Variable types: 0 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 80 rows, 1560 columns, 3120 nonzeros
Variable types: 0 continuous, 1560 integer (1560 binary)

Root relaxation: objective 4.523478e+00, 103 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.97619    0   39          -    4.97619      -     -    0s
H    0     0                       5.149020

{'ofv': 5.0321518831559935,
 'seq': [0,
  36,
  35,
  17,
  21,
  20,
  11,
  6,
  1,
  18,
  10,
  5,
  34,
  15,
  30,
  25,
  31,
  32,
  39,
  19,
  22,
  12,
  33,
  2,
  4,
  37,
  3,
  7,
  9,
  8,
  23,
  29,
  27,
  26,
  28,
  24,
  16,
  38,
  13,
  14,
  0],
 'gap': 0,
 'lowerBound': 5.0321518831559935,
 'upperBound': 5.0321518831559935,
 'runtime': 1.3947792053222656,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [7]:
from tspData.tsp60_0 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 120 rows, 3540 columns and 7080 nonzeros
Model fingerprint: 0xd81b85ee
Variable types: 0 continuous, 3540 integer (3540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 120 rows, 3540 columns, 7080 nonzeros
Variable types: 0 continuous, 3540 integer (3540 binary)

Root relaxation: objective 5.755172e+00, 177 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.75517    0   56          -    5.75517      -     -    0s
     0     0    6.06412    0   20        

{'ofv': 6.212418137291552,
 'seq': [0,
  32,
  42,
  15,
  36,
  31,
  57,
  10,
  2,
  20,
  28,
  12,
  18,
  35,
  11,
  43,
  3,
  26,
  8,
  27,
  4,
  34,
  9,
  39,
  40,
  50,
  44,
  5,
  1,
  53,
  30,
  17,
  59,
  24,
  37,
  29,
  21,
  25,
  52,
  51,
  33,
  49,
  23,
  14,
  22,
  45,
  41,
  48,
  38,
  6,
  16,
  56,
  55,
  13,
  47,
  19,
  46,
  58,
  54,
  7,
  0],
 'gap': 0,
 'lowerBound': 6.212418137291552,
 'upperBound': 6.212418137291552,
 'runtime': 1.8578968048095703,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [8]:
from tspData.tsp60_1 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 120 rows, 3540 columns and 7080 nonzeros
Model fingerprint: 0x10e9b234
Variable types: 0 continuous, 3540 integer (3540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 120 rows, 3540 columns, 7080 nonzeros
Variable types: 0 continuous, 3540 integer (3540 binary)

Root relaxation: objective 5.288051e+00, 145 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.28805    0   30          -    5.28805      -     -    0s
H    0     0                       7.5676

{'ofv': 5.974669109587247,
 'seq': [0,
  30,
  3,
  26,
  22,
  48,
  36,
  16,
  32,
  33,
  27,
  44,
  7,
  11,
  18,
  25,
  39,
  55,
  53,
  46,
  14,
  21,
  23,
  35,
  34,
  52,
  42,
  43,
  19,
  1,
  47,
  29,
  20,
  17,
  24,
  31,
  38,
  8,
  12,
  49,
  51,
  50,
  4,
  41,
  45,
  56,
  13,
  15,
  37,
  6,
  28,
  2,
  54,
  9,
  57,
  40,
  59,
  5,
  10,
  58,
  0],
 'gap': 0,
 'lowerBound': 5.974669109587247,
 'upperBound': 5.974669109587247,
 'runtime': 10.03669548034668,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [9]:
from tspData.tsp60_2 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 120 rows, 3540 columns and 7080 nonzeros
Model fingerprint: 0x6316838d
Variable types: 0 continuous, 3540 integer (3540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 120 rows, 3540 columns, 7080 nonzeros
Variable types: 0 continuous, 3540 integer (3540 binary)

Root relaxation: objective 5.065621e+00, 150 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.51261    0   82          -    5.51261      -     -    0s
     0     0    5.55098    0   22        

{'ofv': 5.688846811299668,
 'seq': [0,
  9,
  39,
  59,
  52,
  2,
  34,
  23,
  42,
  21,
  38,
  11,
  5,
  40,
  15,
  41,
  55,
  31,
  35,
  27,
  51,
  18,
  24,
  54,
  32,
  29,
  4,
  1,
  44,
  26,
  50,
  45,
  13,
  17,
  47,
  43,
  36,
  30,
  33,
  25,
  49,
  37,
  6,
  57,
  10,
  56,
  22,
  3,
  58,
  28,
  16,
  46,
  8,
  19,
  14,
  20,
  53,
  12,
  48,
  7,
  0],
 'gap': 0,
 'lowerBound': 5.688846811299668,
 'upperBound': 5.688846811299668,
 'runtime': 1.204793930053711,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [10]:
from tspData.tsp80_0 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 6320 columns and 12640 nonzeros
Model fingerprint: 0xb6c3821b
Variable types: 0 continuous, 6320 integer (6320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 160 rows, 6320 columns, 12640 nonzeros
Variable types: 0 continuous, 6320 integer (6320 binary)

Root relaxation: objective 6.072384e+00, 211 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.07238    0   22          -    6.07238      -     -    0s
     0     0    6.56016    0   34      

{'ofv': 6.741555142529003,
 'seq': [0,
  16,
  60,
  33,
  39,
  10,
  38,
  59,
  40,
  56,
  25,
  23,
  61,
  3,
  1,
  24,
  29,
  21,
  58,
  9,
  31,
  46,
  13,
  71,
  76,
  12,
  26,
  79,
  65,
  32,
  27,
  30,
  66,
  45,
  47,
  62,
  72,
  5,
  63,
  78,
  19,
  52,
  4,
  42,
  28,
  34,
  67,
  2,
  48,
  70,
  74,
  50,
  53,
  41,
  8,
  54,
  14,
  17,
  37,
  35,
  51,
  7,
  68,
  20,
  6,
  43,
  64,
  44,
  69,
  57,
  49,
  15,
  55,
  18,
  75,
  22,
  11,
  36,
  73,
  77,
  0],
 'gap': 0,
 'lowerBound': 6.741555142529003,
 'upperBound': 6.741555142529003,
 'runtime': 135.80774879455566,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [11]:
from tspData.tsp80_1 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 6320 columns and 12640 nonzeros
Model fingerprint: 0x07006d4d
Variable types: 0 continuous, 6320 integer (6320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 160 rows, 6320 columns, 12640 nonzeros
Variable types: 0 continuous, 6320 integer (6320 binary)

Root relaxation: objective 6.913712e+00, 242 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.91371    0   44          -    6.91371      -     -    0s
     0     0    7.28650    0   28      

{'ofv': 7.430077682282787,
 'seq': [0,
  18,
  8,
  50,
  1,
  76,
  65,
  74,
  52,
  20,
  48,
  28,
  25,
  47,
  39,
  79,
  43,
  53,
  15,
  7,
  16,
  59,
  3,
  22,
  32,
  11,
  56,
  51,
  73,
  75,
  27,
  36,
  21,
  30,
  63,
  38,
  72,
  41,
  58,
  40,
  62,
  26,
  14,
  31,
  34,
  4,
  66,
  55,
  24,
  68,
  9,
  37,
  71,
  70,
  6,
  23,
  45,
  29,
  69,
  57,
  42,
  12,
  5,
  33,
  67,
  10,
  60,
  35,
  46,
  61,
  49,
  13,
  2,
  54,
  78,
  19,
  44,
  17,
  64,
  77,
  0],
 'gap': 0,
 'lowerBound': 7.430077682282787,
 'upperBound': 7.430077682282787,
 'runtime': 22.09620475769043,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [12]:
from tspData.tsp80_2 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 6320 columns and 12640 nonzeros
Model fingerprint: 0x8bece950
Variable types: 0 continuous, 6320 integer (6320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 160 rows, 6320 columns, 12640 nonzeros
Variable types: 0 continuous, 6320 integer (6320 binary)

Root relaxation: objective 6.631769e+00, 247 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.63177    0   40          -    6.63177      -     -    0s
     0     0    6.99025    0   28      

{'ofv': 7.2060802834805395,
 'seq': [0,
  31,
  43,
  35,
  64,
  9,
  5,
  48,
  75,
  54,
  65,
  23,
  10,
  74,
  66,
  78,
  18,
  67,
  37,
  32,
  1,
  77,
  33,
  26,
  51,
  27,
  42,
  2,
  71,
  41,
  60,
  12,
  13,
  28,
  58,
  39,
  63,
  56,
  61,
  15,
  47,
  22,
  52,
  7,
  14,
  44,
  59,
  38,
  34,
  24,
  57,
  72,
  79,
  68,
  53,
  11,
  30,
  69,
  8,
  76,
  49,
  19,
  55,
  70,
  73,
  3,
  25,
  4,
  50,
  16,
  6,
  36,
  17,
  20,
  46,
  62,
  45,
  21,
  40,
  29,
  0],
 'gap': 0,
 'lowerBound': 7.2060802834805395,
 'upperBound': 7.2060802834805395,
 'runtime': 15.28266716003418,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [13]:
from tspData.tsp100_0 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 200 rows, 9900 columns and 19800 nonzeros
Model fingerprint: 0x40c2fa29
Variable types: 0 continuous, 9900 integer (9900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.05s
Presolved: 200 rows, 9900 columns, 19800 nonzeros
Variable types: 0 continuous, 9900 integer (9900 binary)

Root relaxation: objective 7.623033e+00, 290 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.62303    0   82          -    7.62303      -     -    0s
     0     0    7.81291    0  102      

{'ofv': 7.9710615553650035,
 'seq': [0,
  69,
  24,
  95,
  12,
  81,
  74,
  31,
  17,
  80,
  39,
  21,
  19,
  23,
  66,
  42,
  47,
  36,
  94,
  18,
  68,
  51,
  34,
  48,
  49,
  64,
  84,
  40,
  38,
  30,
  67,
  26,
  92,
  62,
  56,
  86,
  41,
  22,
  15,
  33,
  1,
  60,
  89,
  8,
  96,
  5,
  50,
  11,
  54,
  10,
  75,
  99,
  93,
  70,
  29,
  78,
  88,
  57,
  87,
  98,
  59,
  20,
  7,
  65,
  58,
  9,
  83,
  27,
  61,
  45,
  4,
  82,
  71,
  63,
  2,
  53,
  90,
  97,
  72,
  28,
  91,
  76,
  14,
  44,
  43,
  46,
  79,
  73,
  13,
  16,
  52,
  77,
  55,
  35,
  37,
  3,
  85,
  32,
  25,
  6,
  0],
 'gap': 0,
 'lowerBound': 7.9710615553650035,
 'upperBound': 7.9710615553650035,
 'runtime': 209.07829475402832,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [14]:
from tspData.tsp100_1 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 200 rows, 9900 columns and 19800 nonzeros
Model fingerprint: 0xc6faf716
Variable types: 0 continuous, 9900 integer (9900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 200 rows, 9900 columns, 19800 nonzeros
Variable types: 0 continuous, 9900 integer (9900 binary)

Root relaxation: objective 6.807426e+00, 272 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.80743    0   78          -    6.80743      -     -    0s
     0     0    7.34378    0   38      

{'ofv': 7.525137319930366,
 'seq': [0,
  53,
  8,
  69,
  34,
  18,
  82,
  40,
  48,
  94,
  39,
  44,
  76,
  88,
  22,
  30,
  43,
  29,
  35,
  71,
  58,
  12,
  73,
  11,
  33,
  45,
  14,
  9,
  25,
  32,
  79,
  50,
  83,
  24,
  51,
  99,
  62,
  47,
  68,
  98,
  74,
  41,
  87,
  92,
  60,
  52,
  54,
  28,
  86,
  78,
  10,
  55,
  42,
  59,
  77,
  6,
  16,
  90,
  89,
  4,
  26,
  19,
  1,
  85,
  72,
  63,
  21,
  67,
  75,
  38,
  5,
  37,
  61,
  70,
  7,
  57,
  56,
  66,
  95,
  96,
  46,
  3,
  64,
  13,
  36,
  20,
  65,
  17,
  97,
  84,
  15,
  91,
  93,
  31,
  27,
  49,
  23,
  2,
  80,
  81,
  0],
 'gap': 0,
 'lowerBound': 7.525137319930366,
 'upperBound': 7.525137319930366,
 'runtime': 10.622453689575195,
 'fml': 'DFJ_User',
 'serviceTime': 0}

In [15]:
from tspData.tsp100_2 import points
a = list(range(len(points))) 
points_dict=dict(zip(a,points)) 

dist = {(c1, c2): distance(c1, c2) for c1 in range(len(points)) for c2 in range(len(points)) } 
ipTSP(points_dict, dist,fml='DFJ_User',outputFlag=True)

Set parameter LazyConstraints to value 1
Set parameter PreCrush to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 200 rows, 9900 columns and 19800 nonzeros
Model fingerprint: 0xfd7c39eb
Variable types: 0 continuous, 9900 integer (9900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.03s
Presolved: 200 rows, 9900 columns, 19800 nonzeros
Variable types: 0 continuous, 9900 integer (9900 binary)

Root relaxation: objective 7.412660e+00, 275 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.41266    0   82          -    7.41266      -     -    0s
     0     0    7.73406    0   63      

{'ofv': 7.9002944235001085,
 'seq': [0,
  88,
  28,
  62,
  98,
  14,
  67,
  60,
  42,
  50,
  47,
  12,
  8,
  31,
  46,
  22,
  61,
  70,
  44,
  59,
  21,
  58,
  93,
  68,
  73,
  16,
  71,
  86,
  20,
  24,
  95,
  81,
  10,
  39,
  85,
  37,
  65,
  54,
  57,
  56,
  26,
  78,
  92,
  96,
  91,
  51,
  89,
  15,
  64,
  6,
  63,
  2,
  90,
  19,
  80,
  33,
  5,
  94,
  75,
  45,
  55,
  40,
  53,
  29,
  7,
  83,
  36,
  99,
  11,
  52,
  38,
  32,
  77,
  97,
  18,
  79,
  3,
  66,
  30,
  48,
  87,
  1,
  76,
  74,
  17,
  4,
  34,
  69,
  43,
  41,
  25,
  84,
  82,
  35,
  13,
  72,
  9,
  49,
  23,
  27,
  0],
 'gap': 0,
 'lowerBound': 7.9002944235001085,
 'upperBound': 7.9002944235001085,
 'runtime': 60.58584976196289,
 'fml': 'DFJ_User',
 'serviceTime': 0}